In [1]:
#####################################
### Functions for ICC calculation ###
#####################################

def icc(g, e):
    return g/(e+g)

sigma_g = 15.8758 # group variance(s), if multiple, add up to single float 
sigma_e = 10.5788 # residual variance
icc(sigma_g, sigma_e)

0.6001149138524113

In [20]:
###########################################################
#### Function to calculate confidence interval for ICC ####
###########################################################
from scipy.stats import f
def confidence_interval_icc(MS_b, MS_w, df_b, df_w, n, alpha=0.05):
    F = MS_b/MS_w 
    F_u = f.ppf(1-alpha/2, dfn=df_b, dfd=df_w)
    F_l = f.ppf(alpha/2, dfn=df_b, dfd=df_w)
    lower = (F/F_u - 1)/(F/F_u + n - 1)
    upper = (F/F_l - 1)/(F/F_l + n - 1)
    print(f"### Confidence interval: {lower} - {upper}")


MS_b = 40.43722  ## Mean Squares between groups (random effects in anova table)
MS_w = 10.57878  ## Mean squares within group (residual in anova table)
c_n = 1.8808 # constant in anova table, expected mean squares
DF_b = 478 # anova table, random effect(s)
DF_w = 421 # anova table, residual

confidence_interval_icc(MS_b, MS_w, DF_b, DF_w, c_n)

### Confidence interval: 0.5361022806935494 - 0.6567782185548849


In [42]:
###############################
#### Likelihood Ratio Test ####
###############################
from logging import warning
from scipy.stats import chi2

def likelihood_ratio_test(theta, theta_0, df):
    LRT = theta_0 - theta 
    p = 1-chi2.cdf(LRT, df)
    print(f"*** P-value from Chi-Square: {p:.4f} *** \n *** If below 0.05, reject null hypothesis ***")
    if p < 1e-4:
        warning(" P-value extremely low!")
theta_ll = 5359.6 
theta_0_ll = 5374.7 
diff_params = 6
likelihood_ratio_test(theta_ll, theta_0_ll, diff_params)


*** P-value from Chi-Square: 0.0195 *** 
 *** If below 0.05, reject null hypothesis ***


In [43]:
#### Model fitting, check for covariance structure
theta_ll = 29097.6
theta_0_ll = 29136.8 
diff_params = 1
likelihood_ratio_test(theta_ll, theta_0_ll, diff_params)

*** P-value from Chi-Square: 0.0000 *** 
 *** If below 0.05, reject null hypothesis ***


In [46]:
### Model selection: Fixed effects, using ML estimation!
theta_ll = 29062.8
theta_0_ll = 29064.1
diff_params = 1
likelihood_ratio_test(theta_ll, theta_0_ll, diff_params)

*** P-value from Chi-Square: 0.2542 *** 
 *** If below 0.05, reject null hypothesis ***


In [1]:
# # # # # likelihood ratio test
f_value = 5.47
c = 1.2149
((5.47/1.2149)-1)/((f_value/1.2149) + 18.193 - 1)

0.1614362322690226

In [2]:
## upper
((5.47/0.8077)-1)/((5.47/0.8077) + 18.193 - 1)

0.24086126570360766

## Exercise A5

In [2]:
s2_g = 7.8554
s2_e = 31.6176
MS_b = 172.898
MS_w = 31.61
s2_t = s2_g + s2_e
Cn = 18.193
m = 197
def icc(e,g):
    return g/(e + g)
icc(s2_e, s2_g)

0.19900691611987942